In [ ]:
# -*- coding: utf-8 -*-
# 💹 MARKETPULSE - Unified Financial & Sentiment Dashboard (Colab Version)

import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error
from textblob import TextBlob
import yfinance as yf

# ---------------- CONFIG ----------------
tiingo_api_key = "3f69ff9aaf29ca5b8b0daf9340481f323f8481b2"
news_api_key = "aebf914e21f7429d8416bf02c269b305"

default_start_date = "2023-10-16"
default_end_date = "2024-10-16"

# ---------------- Helper Functions ----------------
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

def predict_next_day(model, last_data, scaler, time_step):
    last_data = last_data[-time_step:].reshape(1, time_step, 1)
    prediction = model.predict(last_data)
    return scaler.inverse_transform(prediction)

def get_news(query):
    news_url = f"https://newsapi.org/v2/everything?q={query}&sortBy=publishedAt&language=en&apiKey={news_api_key}"
    response = requests.get(news_url)
    if response.status_code == 200:
        return response.json().get('articles', [])
    else:
        print("⚠️ Error fetching news data.")
        return []

def train_lstm(data_scaled, time_step=25):
    train_size = int(len(data_scaled) * 0.8)
    train, test = data_scaled[:train_size], data_scaled[train_size:]
    X_train, y_train = create_dataset(train, time_step)
    X_test, y_test = create_dataset(test, time_step)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)
    test_predict = model.predict(X_test)
    mae = mean_absolute_error(y_test, test_predict)
    return model, mae

# ---------------- MAIN MENU ----------------
print("💹 Welcome to MarketPulse - Unified Financial Dashboard (Colab Edition)")
print("1️⃣ Stock Prediction")
print("2️⃣ Nifty Index Prediction")
print("3️⃣ Cryptocurrency Prediction")
choice = input("Enter your choice (1/2/3): ").strip()

# ---------------- STOCK ----------------
if choice == "1":
    ticker_symbol = input("Enter Stock Symbol (e.g., AAPL, MSFT, TSLA): ").lower()
    url = f"https://api.tiingo.com/tiingo/daily/{ticker_symbol}/prices"
    params = {'startDate': default_start_date, 'endDate': default_end_date, 'token': tiingo_api_key}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        df = pd.DataFrame(response.json())
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        plt.figure(figsize=(10,5))
        plt.plot(df['close'])
        plt.title(f"{ticker_symbol.upper()} Stock Prices")
        plt.xlabel("Date")
        plt.ylabel("Close Price ($)")
        plt.show()

        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(df[['close']])
        model, mae = train_lstm(data_scaled)
        predicted_tomorrow = predict_next_day(model, data_scaled, scaler, 25)
        print(f"\n📈 Predicted Closing Price for Tomorrow: ${predicted_tomorrow[0,0]:.2f}")
        print(f"MAE: {mae:.5f}")

        print("\n📰 Latest News & Sentiment:")
        articles = get_news(ticker_symbol)
        for article in articles[:5]:
            title = article['title']
            desc = article.get('description', '')
            sent = TextBlob(desc if desc else title).sentiment
            score = "Positive" if sent.polarity > 0 else "Negative" if sent.polarity < 0 else "Neutral"
            print(f"\n🗞️ {title}")
            print(f"Sentiment: {score}")
            print(f"Description: {desc}")
    else:
        print("⚠️ Error fetching stock data.")

# ---------------- NIFTY INDEX ----------------
elif choice == "2":
    index_choice = input("Select Index (50/100): ").strip()
    index_code = "^NSEI" if index_choice == "50" else "^CNX100"

    df = yf.download(index_code, start=default_start_date, end=default_end_date)
    df.reset_index(inplace=True)

    plt.figure(figsize=(10,5))
    plt.plot(df['Close'])
    plt.title(f"Nifty {index_choice} Index Prices")
    plt.xlabel("Date")
    plt.ylabel("Close Price (₹)")
    plt.show()

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df[['Close']])
    model, mae = train_lstm(data_scaled)
    predicted_tomorrow = predict_next_day(model, data_scaled, scaler, 25)
    print(f"\n📈 Predicted Nifty {index_choice} Closing Price for Tomorrow: ₹{predicted_tomorrow[0,0]:.2f}")
    print(f"MAE: {mae:.5f}")

    print("\n📰 Latest Nifty News:")
    articles = get_news("Nifty 50 OR Nifty 100")
    for article in articles[:5]:
        title = article['title']
        desc = article.get('description', '')
        sent = TextBlob(desc if desc else title).sentiment
        score = "Positive" if sent.polarity > 0 else "Negative" if sent.polarity < 0 else "Neutral"
        print(f"\n🗞️ {title}")
        print(f"Sentiment: {score}")
        print(f"Description: {desc}")

# ---------------- CRYPTO ----------------
elif choice == "3":
    crypto_choice = input("Enter Crypto ID (e.g., bitcoin, ethereum): ").lower()
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_choice}/market_chart"
    params = {'vs_currency': 'usd', 'days': '180'}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        prices = data.get('prices', [])
        if not prices:
            print("⚠️ No crypto price data available.")
        else:
            df = pd.DataFrame(prices, columns=['timestamp', 'close'])
            df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('date', inplace=True)
            plt.figure(figsize=(10,5))
            plt.plot(df['close'])
            plt.title(f"{crypto_choice.capitalize()} Price Trend")
            plt.xlabel("Date")
            plt.ylabel("Price (USD)")
            plt.show()

            scaler = MinMaxScaler()
            data_scaled = scaler.fit_transform(df[['close']])
            model, mae = train_lstm(data_scaled)
            predicted_tomorrow = predict_next_day(model, data_scaled, scaler, 25)
            print(f"\n📈 Predicted {crypto_choice.capitalize()} Price for Tomorrow: ${predicted_tomorrow[0,0]:.2f}")
            print(f"MAE: {mae:.5f}")

            print("\n📰 Latest Crypto News:")
            articles = get_news(crypto_choice)
            for article in articles[:5]:
                title = article['title']
                desc = article.get('description', '')
                sent = TextBlob(desc if desc else title).sentiment
                score = "Positive" if sent.polarity > 0 else "Negative" if sent.polarity < 0 else "Neutral"
                print(f"\n🗞️ {title}")
                print(f"Sentiment: {score}")
                print(f"Description: {desc}")
    else:
        print("⚠️ Error fetching crypto data.")

else:
    print("Invalid choice.")
